In [1]:
using Pkg
Pkg.add(["CSV", "DataFrames", "Gurobi", "Distances"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [4]:
using CSV
using DataFrames
using Gurobi
using Random
using Distances
using JuMP
using Gurobi


# Load data
pairs = CSV.read("./data/kidney_data_with_states.csv", DataFrame)
centers = CSV.read("./data/center_small.csv", DataFrame)

# # Take random samples
pair_size = 10
# center_size = 6
Random.seed!(42)
pairs = pairs[shuffle(1:nrow(pairs))[1:min(pair_size, nrow(pairs))], :]
# centers = centers[shuffle(1:nrow(centers))[1:min(center_size, nrow(centers))], :]

n = nrow(pairs)
m = nrow(centers)

println("Number of pairs: ", n)
println("Number of centers: ", m)
    



# Define blood type compatibility scores
blood_type_scores = Dict(
    ("O", "O") => 10, ("O", "A") => 5, ("O", "B") => 5, ("O", "AB") => 3,
    ("A", "A") => 10, ("A", "AB") => 5,
    ("B", "B") => 10, ("B", "AB") => 5,
    ("AB", "AB") => 10
)

# Calculate matching scores
matching_scores = Dict()
for i in 1:n, j in 1:n
    donor = pairs[i, :Donor]
    patient = pairs[j, :Patient]
    matching_scores[(i, j)] = get(blood_type_scores, (donor, patient), 0)
end



# Calculate distances
distances = Dict()
for i in 1:n, k in 1:m
    pair_coords = (pairs[i, :lat], pairs[i, :lng])
    center_coords = (centers[k, :lat], centers[k, :lng])
    distances[(i, k)] = haversine(pair_coords, center_coords, 6371)  # 6371 is Earth's radius in km
end

# Compatibility matrix
comp = Dict()
blood_type_matches = [
    ("O", "O"), ("O", "A"), ("O", "B"), ("O", "AB"),
    ("A", "A"), ("A", "AB"),
    ("B", "B"), ("B", "AB"),
    ("AB", "AB")
]

for i in 1:n, j in 1:n
    donor = pairs[i, :Donor]
    patient = pairs[j, :Patient]
    comp[(i, j)] = (donor, patient) in blood_type_matches ? 1 : 0
end


α = 1.0 / (2 * d_max * n)

Number of pairs: 10
Number of centers: 6


4.620582241596524e-6

In [7]:
# Create model
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, y[1:n, 1:n, 1:m], Bin)  # y[i,j,k] = 1 if donor i donates to patient j through center k
@variable(model, z[1:m], Bin)  # z[k] = 1 if center k is used

# Objective function
@objective(model, Max, 
    # First term: maximize matching scores
    sum(matching_scores[(i,j)] * y[i,j,k] for i in 1:n for j in 1:n for k in 1:m) -
    
    # Second term: minimize travel distance weighted by α
    α * sum((distances[(i,k)] + distances[(j,k)]) * y[i,j,k] for i in 1:n for j in 1:n for k in 1:m)
)

# Constraints
# 1. Donor limit
for i in 1:n
    @constraint(model, sum(y[i,j,k] for j in 1:n for k in 1:m) <= 1)
end

# 2. Patient limit
for j in 1:n
    @constraint(model, sum(y[i,j,k] for i in 1:n for k in 1:m) <= 1)
end

# 3. Compatibility constraint
for i in 1:n, j in 1:n, k in 1:m
    @constraint(model, y[i,j,k] <= comp[(i,j)])
end

# 4. Transplant center capacity
for k in 1:m
    capacity = centers[k, Symbol("Living Donor Transplants in a year")]
    @constraint(model, sum(y[i,j,k] for i in 1:n for j in 1:n) <= capacity * z[k])
end

# 5. Center usage constraint
for k in 1:m
    @constraint(model, z[k] >= sum(y[i,j,k] for i in 1:n for j in 1:n) / (n * n))
end



# Optimize
optimize!(model)

# Print results
if termination_status(model) == MOI.OPTIMAL
    println("\nOptimal Solution Found:")
    for i in 1:n, j in 1:n, k in 1:m
        if value(y[i,j,k]) > 0.5
            println("Donor $i donates to Patient $j through Center $k")
        end
    end
    
    used_centers = [k for k in 1:m if value(z[k]) > 0.5]
    println("\nSelected Transplant Centers: ", [centers[k, Symbol("mix-text_weightBold")] for k in used_centers])
    
    total_exchanges = sum(value(y[i,j,k]) for i in 1:n for j in 1:n for k in 1:m)
    total_travel_distance = sum((distances[(i,k)] + distances[(j,k)]) * value(y[i,j,k]) 
                               for i in 1:n for j in 1:n for k in 1:m)
    
    println("\nTotal Exchanges: ", total_exchanges)
    println("Total Travel Distance: ", round(total_travel_distance, digits=2), " km")
    println("Objective Value: ", round(objective_value(model), digits=2))
else
    println("No optimal solution found.")
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-03
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 632 rows, 606 columns and 3012 nonzeros
Model fingerprint: 0xd5dd3660
Variable types: 0 continuous, 606 integer (606 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+02]
  Objective range  [8e-04, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 79.8315369
Presolve removed 616 rows and 561 columns
Presolve time: 0.00s
Presolved: 16 rows, 45 columns, 90 nonzeros
Found heuristic solution: objective 79.9685929
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: cutoff, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    

In [8]:
# Create DataFrames to store results if optimization was successful
if termination_status(model) == MOI.OPTIMAL
    # Save matching results
    matches_df = DataFrame(
        Donor_ID = Int[],
        Recipient_ID = Int[],
        Center_ID = Int[],
        Donor_Lat = Float64[],
        Donor_Lng = Float64[],
        Recipient_Lat = Float64[],
        Recipient_Lng = Float64[],
        Center_Lat = Float64[],
        Center_Lng = Float64[],
        Distance = Float64[]
    )

    # Collect all successful matches
    for i in 1:n, j in 1:n, k in 1:m
        if value(y[i,j,k]) > 0.5
            total_distance = distances[(i,k)] + distances[(j,k)]
            push!(matches_df, [
                i, j, k,
                pairs[i, :lat], pairs[i, :lng],
                pairs[j, :lat], pairs[j, :lng],
                centers[k, :lat], centers[k, :lng],
                total_distance
            ])
        end
    end

    # Save centers usage
    centers_df = DataFrame(
        Center_ID = Int[],
        Name = String[],
        Lat = Float64[],
        Lng = Float64[],
        Is_Used = Bool[]
    )

    for k in 1:m
        push!(centers_df, [
            k,
            centers[k, Symbol("mix-text_weightBold")],
            centers[k, :lat],
            centers[k, :lng],
            value(z[k]) > 0.5
        ])
    end

    # Save to CSV files
    CSV.write("kidney_matches.csv", matches_df)
    CSV.write("centers_usage.csv", centers_df)
    
    println("Results have been saved to 'kidney_matches.csv' and 'centers_usage.csv'")
end

Results have been saved to 'kidney_matches.csv' and 'centers_usage.csv'
